## Emoji Prediction on Tweets

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import tensorflow.keras

In [12]:
df = pd.read_csv("data/emoji.csv").sample(n=30)
df.dropna(inplace=True)

X = df["Tweet"].values
y = df["Label"].values

In [13]:
emoji_raw = open('data/us_mapping.txt','r',encoding="utf8")

emojis=[]
for sentence in emoji_raw:
    sentence = sentence.rstrip()
    emojis.append(sentence)

    
emoji_dict={}

for e in emojis:
    idx = int(e.split()[0])
    emoji = e.split()[1]
    emoji_dict[idx] = emoji

In [14]:
from keras.preprocessing.sequence import pad_sequences

tokenizer = pickle.load(open("DL_models/tweet_tokenizer",'rb'))

def preprocess_text(X):
    
    max_len=40
    X_seqs = tokenizer.texts_to_sequences(X)
    X_seqs_pd = pad_sequences(X_seqs, truncating="pre", padding="pre", maxlen=max_len)
    
    return X_seqs_pd

In [15]:
from tensorflow import keras

model = keras.models.load_model("DL_models/BLSTM.h5", compile=False)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 128)           8022016   
_________________________________________________________________
bidirectional (Bidirectional (None, 40, 160)           133760    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 40, 160)           154240    
_________________________________________________________________
global_max_pooling1d (Global (None, 160)               0         
_________________________________________________________________
dropout (Dropout)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                10304     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0

In [18]:
X_sequences = preprocess_text(X)
predictions = model.predict_classes(X_sequences)

y_map = [emoji_dict[idx] for idx in y]
predictions = [emoji_dict[idx] for idx in predictions]


results = pd.DataFrame({"Tweet":X})
results["True"] = y_map
results["Predicted"] = predictions


results

,Tweet,True,Predicted
0,witnessed so much talent watching the hip hop ...,😍,😍
1,post show pizzarang nyc pizzarang trumpet trom...,❤️,❤️
2,new years resolution be as tall as him farley...,😘,😘
3,fathers day breakfast with a view marina vari...,😊,😊
4,love what i do and the beauty i create orange...,💜,💜
5,just prowling around during rl grimes set col...,💕,💙
6,amazing fathers day got to spend it with my lo...,❤️,❤️
7,im godzillavyettezilla one world trade center,😂,🇺🇸
8,sunset cliffs beach,😍,😘
9,when sends you pencils for your students …,😁,😁


#    

## Emojify Tweet

In [19]:
from tensorflow import keras

emoji_predict_model = keras.models.load_model("DL_models/BLSTM.h5", compile=False)


def predict_emoji(text, model=emoji_predict_model):
    
    X_sequences = preprocess_text([text])
    predictions = model.predict_classes(X_sequences)
    emoji_idx = predictions[0]
    emoji = emoji_dict[emoji_idx]
    
    return emoji

In [20]:
text = input("Enter tweet \n\n")

print("\n\n Emojified Tweet \n\n")
print(text+" "+predict_emoji(text))

Enter tweet 

Elon is a cool guy @spacex


 Emojified Tweet 


Elon is a cool guy @spacex 😎


In [21]:
text = input("Enter tweet \n\n")

print("\n\n Emojified Tweet \n\n")
print(text+" "+predict_emoji(text))

Enter tweet 

Had great fun this weekend #chrismas


 Emojified Tweet 


Had great fun this weekend #chrismas 🎄
